In [ ]:
import json
import pandas as pd

def process_json(file_path):
    # Đọc file JSON
    with open(file_path, 'r') as file:
        json_data = json.load(file)

    # Danh sách các keys không cần xử lý
    ignored_keys = ['schedule', 'thermal_comfort_data', 'issues_count', 'potential_savings']

    # Tạo danh sách DataFrame tạm thời
    dfs = []

    # Duyệt qua từng key trong dữ liệu JSON
    for key, value in json_data.items():
        if key in ignored_keys or not isinstance(value, dict):
            continue

        # Tạo DataFrame từ từng phần của JSON
        for subkey, subvalue in value.items():
            if isinstance(subvalue, dict):
                temp_df = pd.DataFrame.from_dict(subvalue, orient='index')
                temp_df.index.name = 'timestamp'
                temp_df.reset_index(inplace=True)
                temp_df.columns = ['timestamp', f'{key}_{subkey}']
                dfs.append(temp_df)

    # Hợp nhất các DataFrame tạm thời dựa trên cột timestamp
    final_df = dfs[0]
    for df in dfs[1:]:
        final_df = pd.merge(final_df, df, on='timestamp', how='outer')

    return final_df


In [ ]:
# Áp dụng cho file data_prepare_1.json
file_path_1 = '../dataset/data_prepare_2.json'
df1 = process_json(file_path_1)

# Áp dụng cho file data_prepare_8.json
file_path_8 = '../dataset/data_prepare_8.json'
df8 = process_json(file_path_8)

In [ ]:
# Hiển thị DataFrame đầu tiên để kiểm tra
df1.head()

In [ ]:
df8.head()

In [ ]:
parquet_file_path = '../dataset/data_prepare_2.parquet'
df1.to_parquet(parquet_file_path, index=False)

In [ ]:
parquet_file_path = '../dataset/data_prepare_8.parquet'
df8.to_parquet(parquet_file_path, index=False)